<h1>IDEA: Un algoritmo de clave privada</h1>

<h2> Cifrado de un mensaje, dados los siguientes datos:</h2>
<li> Mensaje="En un lugar de la Mancha, de cuyo nombre no quiero acordarme"</li>
<li> Clave privada (K) => 128 bits ="MARCOSDIEGOSANTI"

<h2>1. Obtención de los valores binarios del mensaje y la clave </h2>

In [1]:
#mensaje_claro="En un lugar de la Mancha, de cuyo nombre no quiero acordarme"
mensaje_claro="6E637F8A8B8C8394"
#La clave tiene que usar un alfabeto que contenga menos de 128 bits
#clave_privada="MARCOSDIEGOSANTI"
clave_privada="11112222333344445555666677778888"
alf="AÁBCDEÉFGHIÍJKLMNÑOÓPQRSTUÚVWXYZ.,;:()¿?¡!-0123456789aábcdeéfghiíjklmnñoópqrstuúvwxyz "

def cadena_a_binario(cadena):
    #Conversion a binario de 8 bits
    cadena_binaria = "".join(format(int(x, 16), '04b') for x in cadena)
    #cadena_binaria = "".join(format(alf.index(x), '08b') for x in cadena)
    return cadena_binaria

mensaje_binario = cadena_a_binario(mensaje_claro)
clave_binaria = cadena_a_binario(clave_privada)

#var = 0b01

#aux = clave_binaria[25:]+clave_binaria[:25]
#lista = [int(aux[x:x+4], 2) for x in range(0,128,4)]
#print(lista)
#print(aux)
#print(len(aux))
#print(var<<1)
print(mensaje_binario)


0110111001100011011111111000101010001011100011001000001110010100


<h2>2. Se divide el mensaje binario en bloques de 64 bits. Cada bloque se divide a su vez en 4 sub-bloques de 16 bits </h2>


In [2]:
def binario_a_bloques(cadena, tam_bloque):
    lista_bloques = [cadena[x:x+tam_bloque] for x in range(0,len(cadena), tam_bloque)]
    len_ultimo_bloque = len(lista_bloques[-1])
    #Relleno con 0s si no se alcanza el tamaño del bloque
    if(len_ultimo_bloque!=tam_bloque):
        lista_bloques[-1] = lista_bloques[-1]+(tam_bloque-len_ultimo_bloque)*"0"
    return lista_bloques

#Mensaje dividido en bloques de 64 bits, que tendrá la forma [ab,cd]
mensaje_dividido = binario_a_bloques(mensaje_binario, 64)

#Cada sub-bloque lo divido en 4 bloques de 16 bits, guardo el resultado final, que tendrá la forma [[a,b],[c,d]]
mensaje_bloques = []
for bloque in mensaje_dividido:
    mensaje_bloques.append(binario_a_bloques(bloque, 16))

print(int(mensaje_bloques[0][0],2))



28259


<h2>3. Se calculan las claves intermedias. Serán un total de 52 subclaves de 16 bits, que se emplearán para cifrar el mensaje</h2>

In [3]:
 #Recibe un string de valores binarios
 def generar_claves_intermedias_cifrado(clave_inicial):
     # lista de una dimensión, que contendrá las 52 subclaves
     lista_claves = []
     #La agrupación será de 6 en 6 subclaves. Las 8 primeras son la clave inicial.
     lista_claves.extend(binario_a_bloques(clave_inicial, 16))

     while(len(lista_claves)<52):
         #Desplazar 25 bits a la izquierda la clave inicial
         clave_inicial = clave_inicial[25:]+clave_inicial[:25]
         #Bloques de la clave actualizada
         lista_claves.extend(binario_a_bloques(clave_inicial, 16))

     # lista_claves crece en múltiplos de 8 y es por ello que debemos quedarnos solo con los 52 primeros valores
     lista_claves = lista_claves[:52]
     #print(lista_claves)
     return lista_claves

claves_intermedias = generar_claves_intermedias_cifrado(clave_binaria)
#print(claves_intermedias)


<h2>4. Aplico el algoritmo de cifrado a cada bloque del mensaje</h2>

In [4]:
def calcular_iteracion(bloque_numerico, subclaves):
        print("Mensaje:"," ".join(bin(x) for x in bloque_numerico),"Claves:", subclaves)
        #Paso 1
        var1 = (bloque_numerico[0] * subclaves[0])%(2**16+1)

        print("Paso 1:", bin(var1))

        #Paso 2
        var2 = (bloque_numerico[1] + subclaves[1])%(2**16)

        print("Paso 2:", bin(var2))

        #Paso 3
        var3 = (bloque_numerico[2] + subclaves[2])%(2**16)

        print("Paso 3:", bin(var3))

        #Paso 4
        var4 = (bloque_numerico[3] * subclaves[3])%(2**16+1)

        print("Paso 4:", bin(var4))

        #Paso 5 (XOR entre 1 y 3)
        var5 = var1 ^ var3

        print("Paso 5:", bin(var5))

        #Paso 6 
        var6 = var2 ^ var4

        print("Paso 6:", bin(var6))

        #Paso 7
        var7 = (var5 * subclaves[4])%(2**16+1)

        print("Paso 7:", bin(var7))

        #Paso 8
        var8 = (var6 + var7)%(2**16)

        print("Paso 8:", bin(var8))

        #Paso 9
        var9 = (var8 * subclaves[5])%(2**16+1)
        
        print("Paso 9:", bin(var9))

        #Paso 10
        var10 = (var7 + var9)%(2**16)

        print("Paso 10:", bin(var10))

        #Paso 11
        var11 = var1 ^ var9

        print("Paso 11:", bin(var11))
        print("Paso 1:", len(bin(var11)))

        #Paso 12
        var12 = var3 ^ var9

        print("Paso 12:", bin(var12))
        print("Paso 1:", len(bin(var12)))

        #Paso 13
        var13 = var2 ^ var10

        print("Paso 13:", bin(var13))
        print("Paso 1:", len(bin(var13)))

        #Paso 14
        var14 = var4 ^ var10

        print("Paso 14:", bin(var14))
        print("Paso 1:", len(bin(var14)))

        return [var11, var12, var13, var14]


def lista_binaria_a_decimal (lista_binaria):
    return [int(elem,2) for elem in lista_binaria]

def aplicar_cifrado_bloque(bloque_msj, claves_intermedias):
    #La primera iteración usara los valores numericos del bloque del mensaje
    resultado_iteracion = lista_binaria_a_decimal(bloque_msj)
    #resultado_iteracion = [int(bloque_msj[0],2), int(bloque_msj[1],2), int(bloque_msj[2],2), int(bloque_msj[3],2)]
    iteraciones = []
    for i in range(0,8):
        #En cada iteración se cogen 6 subclaves
        subclaves = lista_binaria_a_decimal(claves_intermedias[i*6:(i*6)+6])
        resultado_iteracion = calcular_iteracion(resultado_iteracion, subclaves)
        iteraciones.extend(resultado_iteracion)
        print(resultado_iteracion)
        #Se retroalimenta el algoritmo usando como bloque de entrada la permutación de los elementos centrales de la anterior iteración
        resultado_iteracion = [resultado_iteracion[0], resultado_iteracion[1], resultado_iteracion[2], resultado_iteracion[3]]

    print("Iteraciones",iteraciones)

    #Último paso 
    #Ultimas 4 subclaves
    subclaves = lista_binaria_a_decimal(claves_intermedias[48:52])

    resultado_iteracion = [resultado_iteracion[0], resultado_iteracion[2], resultado_iteracion[1], resultado_iteracion[3]]
    var15 = (resultado_iteracion[0] * subclaves[0])%(2**16+1)
    var16 = (resultado_iteracion[1] + subclaves[1])%(2**16)
    var17 = (resultado_iteracion[2] + subclaves[2])%(2**16)
    var18 = (resultado_iteracion[3] * subclaves[3])%(2**16+1)

    resultado = [var15, var16, var17, var18]

    print("Resultado",resultado)

    for elem in resultado:
        num = format(elem, "016b")
        #print(len(num))
        print(hex(int(num[:4],2)),end=' ')
        print(hex(int(num[4:8],2)),end=' ')
        print(hex(int(num[8:12],2)),end=' ')
        print(hex(int(num[12:16],2)),end=' ')

    return resultado


def cifrar_mensaje(mensaje_bloques, claves_intermedias):
    msj_cifrado = []

    for bloque in mensaje_bloques:
        msj_cifrado.append(aplicar_cifrado_bloque(bloque, claves_intermedias))

    print(msj_cifrado)

cifrar_mensaje(mensaje_bloques, claves_intermedias)

Mensaje: 0b110111001100011 0b111111110001010 0b1000101110001100 0b1000001110010100 Claves: [4369, 8738, 13107, 17476, 21845, 26214]
Paso 1: 0b1110000000111000
Paso 2: 0b1010000110101100
Paso 3: 0b1011111010111111
Paso 4: 0b10000000111010
Paso 5: 0b101111010000111
Paso 6: 0b1000000110010110
Paso 7: 0b1011001010001
Paso 8: 0b1001011111100111
Paso 9: 0b101001101001000
Paso 10: 0b110100110011001
Paso 11: 0b1011001101110000
Paso 1: 18
Paso 12: 0b1110110111110111
Paso 1: 18
Paso 13: 0b1100100000110101
Paso 1: 18
Paso 14: 0b100100110100011
Paso 1: 17
[45936, 60919, 51253, 18851]
Mensaje: 0b1011001101110000 0b1110110111110111 0b1100100000110101 0b100100110100011 Claves: [30583, 34952, 17510, 26248, 34986, 43724]
Paso 1: 0b10010101010100
Paso 2: 0b111011001111111
Paso 3: 0b110010011011
Paso 4: 0b1111001100011011
Paso 5: 0b10100111001111
Paso 6: 0b1000010101100100
Paso 7: 0b1010010100100101
Paso 8: 0b10101010001001
Paso 9: 0b1100001011001100
Paso 10: 0b110011111110001
Paso 11: 0b1110011110011000